In [2]:
import urllib.request
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common import exceptions
import pymysql
from konlpy.tag import Kkma
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np
import kss


conn = pymysql.connect(host='newdb.c7p2ncpgik7h.ap-northeast-2.rds.amazonaws.com', user='admin', password='1dlckdals!',
                       db='TEST1', charset='utf8')
curs = conn.cursor()
session = requests.Session()
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit 537.36 (KHTML, like Gecko) Chrome",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
}


chrome_options=webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

#driver = webdriver.Chrome(r"/home/capston/chromedriver",chrome_options=chrome_options)
#driver = webdriver.Chrome(r"C:\Users\LCM\Downloads\chromedriver_win32 (2)\chromedriver.exe")
#driver = webdriver.Chrome(r"C:\Users\LCM\Downloads\chromedriver_win32\chromedriver.exe")
driver = webdriver.Chrome(r"C:\Users\seenw\Downloads\chromedriver_win32\chromedriver.exe")


class SentenceTokenizer(object):
    def __init__(self):
        self.kkma = Kkma()
        self.okt = Okt()
        # 불용어 불러오기
        self.stopwords = [line.rstrip('\n') for line in open('stopwords_korean2.txt', encoding = 'utf-8')]
    
    def text2sentences(self, text):
        #sentences = self.kkma.sentences(text)\
        sentences = kss.split_sentences(text)
        sentences = sentences[0:len(sentences)-2]
        
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
                
        return sentences
    
    def get_nouns(self, sentences):
        nouns = []
        for sentence in sentences:
            if sentence != '':
                nouns.append(' '.join([noun for noun in self.okt.nouns(str(sentence))
                    if noun not in self.stopwords and len(noun) > 1]))
                
        return nouns


class GraphMatrix(object):
    def __init__(self):
        self.tfidf = TfidfVectorizer()
        self.cnt_vec = CountVectorizer()
        self.graph_sentence = []
        
    def build_sent_graph(self, sentence):
        tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        self.graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
        return self.graph_sentence
    
    def build_words_graph(self, sentence):
        cnt_vec_mat = normalize(self.cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
        vocab = self.cnt_vec.vocabulary_
        return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}
    
    
class Rank(object):
    def get_ranks(self, graph, d=0.85): # d = damping factor
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id, id] = 0 # diagonal 부분을 0으로
            link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
            if link_sum != 0:
                A[:, id] /= link_sum
                
            A[:, id] *= -d
            A[id, id] = 1
            
        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
        return {idx: r[0] for idx, r in enumerate(ranks)}

    
    
class TextRank(object):
    def __init__(self, text):
        self.sent_tokenize = SentenceTokenizer()
        self.sentences = self.sent_tokenize.text2sentences(text)
            
        self.nouns = self.sent_tokenize.get_nouns(self.sentences)
        self.graph_matrix = GraphMatrix()
        self.sent_graph = self.graph_matrix.build_sent_graph(self.nouns)
        self.words_graph, self.idx2word = self.graph_matrix.build_words_graph(self.nouns)
        self.rank = Rank()
        self.sent_rank_idx = self.rank.get_ranks(self.sent_graph)
        self.sorted_sent_rank_idx = sorted(self.sent_rank_idx, key=lambda k: self.sent_rank_idx[k], reverse=True)
        self.word_rank_idx = self.rank.get_ranks(self.words_graph)
        self.sorted_word_rank_idx = sorted(self.word_rank_idx, key=lambda k: self.word_rank_idx[k], reverse=True)
        #print(self.nouns)
        
    def summarize(self, sent_num=3):
        summary = []
        index=[]
        for idx in self.sorted_sent_rank_idx[:sent_num]:
            index.append(idx)
        
        index.sort()
        for idx in index:
            summary.append(self.sentences[idx])
        
        return summary
    
    def keywords(self, word_num=10):
        rank = Rank()
        rank_idx = rank.get_ranks(self.words_graph)
        sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)
        
        keywords = []
        index=[]
        for idx in sorted_rank_idx[:word_num]:
            index.append(idx)
            
        #index.sort()
        for idx in index:
            keywords.append(self.idx2word[idx])
            
        return keywords


class newsCrawler:
    def __init__(self):
        self.titleList=[]
        self.contentsList=[]
        self.imageList=[]
        self.dateList=[]
    # 네이버 뉴스홈
    def mainCrawl(self):    
        # 정치=100 경제=101 사회=102 생활/문화=103 세계=104 IT/과학=105
        for category in range(100, 106):
            main_url = "https://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1="+str(category)
            driver.get(main_url)
            
            # '헤드라인 더보기' 버튼이 있다면 누르기       
            self.showMore()
            driver.implicitly_wait(3)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')  
            
            # 헤드라인 가져오기
            self.subCrawl(soup,category)
        driver.quit()
    
    # 더보기버튼 클릭
    def showMore(self):
        try:
            while True:
                print("더보기")
                driver.find_element_by_xpath('//*[@id="main_content"]/div/div[2]/div[2]/div/a').click()
                driver.implicitly_wait(0.5)
        except exceptions.ElementNotVisibleException:
            pass
        except Exception:
            pass
    
    

    # 헤드라인 뉴스 크롤링
    def subCrawl(self, soup,category):
        # 모든 헤드라인 뉴스 저장
        articles = soup.find_all('div', {'class': 'cluster_group _cluster_content'})
        
        for i in range(len(articles)):
            # 각 뉴스 본문에 있는 이미지와 이미지URL를 저장할 리스트
            images=[]
            imagesURL="NO IMAGE"

            temp = articles[i].find_all('div', {'class': 'cluster_text'})[0]

            conURL = temp.a.get('href')
            html2 = session.get(conURL,headers=headers).content
            soup2 = BeautifulSoup(html2, 'html.parser')

            summary = soup2.find('strong', {'class':'media_end_summary'})
            if summary==None:
                summary=""
            else:
                summary=summary.text
            
            content = soup2.find('div', id= "articleBodyContents").text.replace("\n"," ").replace(str(summary),"").replace("\t"," ").replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}"," ")
            title=soup2.find('h3',id="articleTitle").text
            
            # 기사 본문이 10문장이하라면 저장하지 않는다.
            if(len(kss.split_sentences(content)) <= 10):
                continue;
                
            #print(title)
            date=soup2.find('span', {'class','t11'}).text
            #print(contents)
            #print(date)
            # 이미지 추출
            images=soup2.find_all('span', {'class','end_photo_org'})
            
            for i in range(len(images)):
                imagesURL=(images[i].find("img")["src"])
            
            self.saveToDB(str(title),str(content),str(imagesURL),str(date),str(category))


        # DB에 저장
        #self.saveToDB(self.titleList,self.contentsList,self.imageList,self.dateList)

    def saveToDB(self,title,content,imagesURL,date,category):
        content=content.replace("'","")
        sum = TextRank(content)
        
        # content로 
        
        
        content=sum.summarize(7)
        count=1
        # 요약문이 7문장보다 짧을 경우
        if len(content)<7:
            return 2
        
        for i in content:
            # 요약문에 빈문장이 포함된 경우
            if i=="":
                print('중지됨')
                return 3
            print(i)
            print(count)
            print("\n")
            count=count+1
        title = title.replace("'","")

        # SQL문 실행
        sql = "USE TEST1"
        curs.execute(sql)
        '''
        CREATE TABLE NEWS4 (TITLE CHAR(200) NOT NULL,
        CONTENT1 TEXT NOT NULL,
        CONTENT2 TEXT NOT NULL,
        CONTENT3 TEXT NOT NULL,
        CONTENT4 TEXT NOT NULL,
        CONTENT5 TEXT NOT NULL,
        CONTENT6 TEXT NOT NULL,
        CONTENT7 TEXT NOT NULL,
        DATE VARCHAR(40) NOT NULL,
        CATEGORY VARCHAR(40),
        COUNT int NOT NULL AUTO_INCREMENT,
        IMAGE TEXT NOT NULL,
        CONSTRAINT PLAYER_PK PRIMARY KEY (COUNT));
        '''

        #sql3="insert into NEWS3(title,content,date,category,image) VALUES(" +title+ ',' +content+ ',' +date+ ',' +category+ ',' +imagesURL+ ");"
        sql3="""insert into NEWS7(title,content1,content2,content3,content4,content5,content6,content7,date,category,image) VALUES('%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s');"""%(title,content[0],content[1],content[2],content[3],content[4],content[5],content[6],date,category,imagesURL)
        curs.execute(sql3)
        conn.commit()
        print("db updated!")

        return 1


Crawl=newsCrawler() 
Crawl.mainCrawl()
conn.commit()
curs.close()
print('done')

더보기
더보기
"부동산 거래 사전신고제 도입 검토…부당이익 3~5배 환수 조치"강력한 LH 개혁방안 마련키로…"토지주택·주거복지로 나뉠 수도"고위당정협의회에서 발언하는 정세균 총리(서울=연합뉴스) 하사헌 기자 = 정세균 국무총리가 19일 오전 국회 더불어민주당 당대표회의실에서 열린 제3차 고위당정협의회에서 발언하고 있다.
1


toadboy@yna.co.kr    (서울=연합뉴스) 고동욱 김동호 기자 = 더불어민주당과 정부는 19일 공직자 부동산 재산등록제를 전면 확대하는 등 부동산 투기 근절을 위한 관리·감독을 강화한다고 밝혔다.
2


김태년 당대표 직무대행 겸 원내대표는 이날 국회에서 열린 고위 당정청 협의회에서 "한국토지주택공사(LH) 등 부동산 관련 업무를 하는 공직자는 재산등록을 의무화하고, 향후 공무원·공공기관·지자체·지방 공기업을 포함한 모든 공직자로 재산등록을 확대하는 방안을 검토하겠다"며 "부동산 거래시 사전신고제 도입도 적극 검토하겠다"고 말했다.
3


김 대표 대행은 "부당이익이 있다면 3∼5배를 환수조치하겠다"고 강조했다.
4


정세균 국무총리는 "앞으로 신규 택지를 발표할 때 토지 소유 현황이나 거래 관계 사전조사를 통해 더는 투기 논란이 발생하지 않도록 철저히 관리하겠다"며 "부동산 거래 분석 전담 조직도 확대해 조기에 출범시키겠다"고 말했다.
5


검·경간 고위급·실무급 수사협력 체계를 상시 가동해 범죄수익의 규명과 박탈 등을 돕고, 필요한 경우 검찰이 직접 수사에 참여하도록 했다.
6


최인호 수석대변인은 "협력체계가 구축되면 수사가 구체화되면서 검찰이 직접 수사할 케이스도 정해질 것"이라며 "중대 범죄 케이스가 있을 것이다. 수사 성과를 내놓으라는 국민 요구에 칸막이를 쳐서는 안 된다"고 말했다.
7


db updated!
왼쪽은 이상진 센터장. 2021.2.25/뉴스1    고(故) 박원순 전 서울시장 성폭력 피해자에 대한 피해호소인 호칭 사용을 주도한 것으로 알려진 고민정·진선미·남인순 더불어민주당 의원이 18일 

김영훈 기획재정부 경제분석과장이 19일 세종시 정부세종청사에서 2021년 3월 최근경제동향 주요 내용을 설명하고 있다.
1


기획재정부는 19일 발간한 ‘최근 경제동향(그린북 3월호)’에서 “최근 우리 경제는 수출과 투자 등의 개선세가 이어지는 가운데 고용 감소폭이 축소되었으나, 코로나19 확산 등으로 내수 부진이 지속되는 상황”이라고 총평했다.
2


다만 지난해 7월부터 올해 2월까지 8개월 연속으로 실물경제의 ‘불확실성’을 언급했다가 9개월 만에 불확실성이라는 표현을 뺐다.
3


김영훈 기재부 경제분석과장은 이날 브리핑에서 “최근 실물경제 흐름을 보면 수출과 투자가 뚜렷한 개선세를 이어가고 있으며 내수도 2월 이후 부진의 폭이 완화되는 상황”이라며 “이런 흐름을 볼 때 단기간 내에 실물경제 지표가 추가 하락할 가능성은 낮아진 게 아닌가 판단한다”고 설명했다.
4


이어 “최근 주요 전망기관들이 글로벌 경제뿐 아니라 우리 경제의 성장 전망을 상향하고 있는 것도 대외 의존도가 높은 우리나라로서는 긍정적인 사인”이라며 “수출 측면에서의 회복세가 더 크게 확대될 여력이 있다”고 덧붙였다.
5


뉴시스    백화점 매출액은 39.5% 급증해 정부가 그린북을 발간하며 모니터링을 시작한 2005년 이후 최대 증가폭을 기록했다.
6


2월 취업자는 1년 전보다 47만3000명 감소하며 지난해 3월 이후 12개월 연속 감소를 기록했다.
7


db updated!
화물노선 타이베이 등 총 3개로 여객수송 부진 속 매출증대 전략CGV 손잡고 컨셉상영회도 진행제주항공이 지난 18일부터 오는 24일까지 CGV와 함께 여행과 영화를 결합한 `컨셉 상영회`를 진행한다고 19일 밝혔다.
1


[사진 제공 = 제주항공] 제주항공이 지난 18일부터 인천~베트남 호치민 화물 노선 운항을 시작했다고 19일 밝혔다.
2


해당 화물 노선은 인천공항에서 매주 화, 목, 토요일 밤 10시10분(현지시간 기준)에 출발해 호치민공항에 이튿날 새벽 1시50분에 도착한다.
3


호치민공

안팔리던 그림 NFT로 만들자수천~수억원대에 거래 기적 비플 JPG파일은 783억에 낙찰오프라인보다 작품가 10배 높아 데미안 허스트 등 유명 작가 뛰어들고크리스티·소더비·서울옥션도 진출 4700만원에 팔린 미스터 미상 NFT작품 #04. Birth of Mr Misang. <사진제공=미스터 미상> 국내 디지털 아티스트 미스터 미상은 자동차와 게임회사 주문으로 일러스트레이션과 애니메이션 등을 제작해 생업을 이어나갔다.
1


최근 이더리움 시세가 200만원대로 폭등하면서 NFT 미술품 가격도 치솟고 있다.
2


지난 11일 뉴욕 크리스티 온라인  783억원에 팔린 비플 NFT 콜라주 작품 매일 첫 5000일 중 한 작품. <사진제공=크리스티 경매>  783억원에 팔린 비플 NFT 콜라주 작품 매일 첫 5000일 중 한 작품.<사진제공=크리스티 경매> 경매에서 디지털 아티스트 비플이 제작한 NFT 콜라주 작품 매일: 첫 5000일이 무려 6930만달러(약 783억원)에 낙찰되자 세계 미술계가 발칵 뒤집어졌다.
3


6억원에 팔린 마리킴 NFT작품 Missing and found. <사진제공=피카프로젝트> 국내 NFT 플랫폼 디파인아트에서도 인기 작가 마리킴 NFT 작품 Missing and found(2021)가 지난 18일 경합 끝에 6억원(288이더리움)에 팔렸다.
4


마리킴은 "지금 NFT 작품을 파는 것도 한 발 늦은 느낌이다"며 "비트코인이나 이더리움으로 돈을 번 밀레니얼 세대(1981~1996년생)들이 주로 NFT 작품을 구입하며, 대부분 작가들이 오프라인 작품가보다 10배 정도 높은 가격에 팔고 있다"고 말했다.
5


크리스티가 비플 경매로 먼저 치고 나가자 소더비도 내달 디지털 아티스트 팍(Pak) 작품 경매를 열 예정이다.
6


NFT가 디지털 아트 판로를 열고 소장자와 거래 가격 기록이 남아 미술 시장 투명성을 높이는 점은 긍정적이지만 작품 가격에 거품이 심하다는 지적도 있다.
7


db updated!
송산리고분군 전경. 문화재청

db updated!
한인 여성 4명 등 모두 8명이 희생된 미국 조지아주 애틀랜타 연쇄 총격이 벌어진 지 이틀 뒤인 18일(현지시간) 세 군데 범행장소 가운데 한인 여성들이 숨을 거둔 애틀랜타 시내 두 스파 업소 근처에 마련된 임시 추모 터에 더 많은 꽃과 아시아인 증오범죄를 끝내자는 팻말이 눈에 띈다.
1


애틀랜타 AFP 연합뉴스한인 여성 4명을 포함해 8명의 목숨을 앗아간 미국 조지아주 애틀랜타 총격 사건과 관련, 경찰이 한인 희생자들의 신원을 아직 밝힐 수 없는 단계라고 18일(이하 현지시간) 설명했다.
2


햄프턴 부서장은 피해자들의 가장 가까운 친족에게 통보하는 데 어려움을 겪고 있으며 한국 영사관과 협력해 노력하고 있다고 말했다.
3


그는 “100% (친족에게) 통보되면 곧 그것이 공개될 것”이라고 말했다.
4


햄프턴 부서장은 애틀랜타 경찰이 희생자들의 사랑하는 사람에게 먼저 통보가 이뤄지도록 기다리고 있다고 말했다.
5


로버트 에런 롱(21)이 지난 16일 저지른 연쇄 총격으로 애틀랜타 근처 체로키 카운티 악워스의 마사지숍에서 4명이 숨지고 1명이 크게 다쳤으며 애틀랜타 시내 스파 두 곳에서 한인 여성 4명이 사망했다.
6


체로키 카운티 보안관실은 관할 지역에서 숨진 희생자 4명과 부상자 1명의 신원을 전날 공개해 이틀이 지나도록 한인 희생자 4명의 신원이 공개되지 않은 것과 대조를 이뤘다.
7


db updated!
더보기
더보기
KT 제2노조인 KT새노조는 19일 "영업이익과 연동한 성과급 체제로 개편해야 한다"고 사측에 요구했다.
1


KT새노조는 성명서에서 "합리적으로 기업의 성과를 측정하고 그에 맞춰 공정하게 분배하자는 것이 젊은 직원들의 요구지만 KT의 성과급 체계는 사실상 공기업 시절 정기 상여금에서 명칭만 바뀌었다"고 했다.
2


KT새노조는 "성과 측정의 원칙이 낡고 불투명한 상태에서 성과 배분 시스템이 상대 평가"라며 "전사적 성과 관리 대신 내부경쟁과 줄세우기 문화가 기업을 짓누르고 있다"고 지적했다.

In [11]:
print(imagesURL)

NameError: name 'imagesURL' is not defined